In [1]:
%matplotlib inline

import os
import time

import osmnx as ox

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import Point, Polygon

import multiprocessing as mp



ModuleNotFoundError: No module named 'osmnx'

In [3]:
import numpy as np

import rasterio


In [7]:
out = '/Users/d3y010/repos/github/cerf/cerf/data/hifld_substation_230kv_dist_km.npy'

In [4]:
f = '/Users/d3y010/repos/github/cerf/cerf/data/hifld_substation_230kv_dist_m.tif'

# load distance to suitable transmission infrastructure raster
with rasterio.open(f) as src:
    ic_dist_km_arr = src.read(1) / 1000  # convert meters to km




In [8]:
np.save(out, ic_dist_km_arr)


In [10]:
np.max(ic_dist_km_arr)

1151.4026

# File setup

In [2]:
fishnet_file = '/Users/d3y010/projects/cerf/data/suitability/prep_v2/shp/fishnet_1km_albers.shp'

target_data_file = '/Users/d3y010/projects/cerf/data/suitability/prep_v2/shp/AL_Wetlands.shp'

# Read in target data and reproject to the CRS of the fishnet data

In [3]:
# read in target polygon data and only keep the geometry
target_gdf = gpd.read_file(target_data_file)[['geometry']]

# only read in 1 row of the fishnet just to get the coordinate system; 
#  saves time instead of importing the whole thing at this point
fishnet_crs = gpd.read_file(fishnet_file, rows=1).crs

# reproject the target data; this takes a while
target_gdf = target_gdf.to_crs(fishnet_crs)



# Read in the fishnet polygons that intersect the bounding box of the target data.  This is done to speed up the load process.

### Construct a geodataframe from the bounding box coordinates


In [4]:
bbox = target_gdf.total_bounds

p1 = Point(bbox[0], bbox[3])
p2 = Point(bbox[2], bbox[3])
p3 = Point(bbox[2], bbox[1])
p4 = Point(bbox[0], bbox[1])

np1 = (p1.coords.xy[0][0], p1.coords.xy[1][0])
np2 = (p2.coords.xy[0][0], p2.coords.xy[1][0])
np3 = (p3.coords.xy[0][0], p3.coords.xy[1][0])
np4 = (p4.coords.xy[0][0], p4.coords.xy[1][0])

bbox_polygon = Polygon([np1, np2, np3, np4])


### Read in the features from the fishnet data that are in the bounding box of the target data

In [5]:
fishnet_gdf = gpd.read_file(fishnet_file, bbox=bbox_polygon)

In [6]:
def get_data(fishnet_gdf, target_gdf, fishnet_id):
    """Return the total area of all target features in a single fishnet polygon.
    
    :returns:           The total area in square meters of the target features that are in the 
                        fishnet polygon after being clipped
            
    """
    
    # subset the fishnet data frame by the target id
    xdf = fishnet_gdf.loc[fishnet_gdf['id'] == fishnet_id].copy()
    
    # create R-Tree spatial index
    sindex = target_gdf.sindex
    
    # find possible matches
    possible_matches_index = list(sindex.intersection(xdf.geometry.bounds.values[0]))
    possible_matches = target_gdf.iloc[possible_matches_index].copy()
    
    # account for invalid geometries
    possible_matches['geometry'] = possible_matches.geometry.buffer(0)
    
    # clip the target data to the fishnet polygon of interest
    clipped_gdf = gpd.clip(possible_matches, mask=xdf)
    
    # recalculate the area of the clipped features
    clipped_gdf['sqm'] = clipped_gdf.area
    
    return {fishnet_id: clipped_gdf.sqm.sum()}
    
    
    
    

In [7]:
def get_data_apply(xdf, target_gdf):
    """Return the total area of all target features in a single fishnet polygon.
    
    :returns:           The total area in square meters of the target features that are in the 
                        fishnet polygon after being clipped
            
    """
    
    # create R-Tree spatial index
    sindex = target_gdf.sindex
    
    # find possible matches
    possible_matches_index = list(sindex.intersection(xdf.geometry.bounds))
    possible_matches = target_gdf.iloc[possible_matches_index].copy()
    
    # repair geometry if broken
    try:
    
        # clip the target data to the fishnet polygon of interest
        clipped_gdf = gpd.clip(possible_matches, mask=xdf.geometry)
        
    except:

        # account for invalid geometries
        possible_matches['geometry'] = possible_matches.geometry.buffer(-0.000001)

        # clip the target data to the fishnet polygon of interest
        clipped_gdf = gpd.clip(possible_matches, mask=xdf.geometry)
    
    # recalculate the area of the clipped features
    clipped_gdf['sqm'] = clipped_gdf.area
    
    return clipped_gdf.sqm.sum()

In [8]:
def getit(fishnet_gdf_subset, target_gdf):
    
    fishnet_gdf_subset['sqm_target'] = fishnet_gdf_subset.apply(lambda x: get_data_apply(x, target_gdf), axis=1)
    
    return fishnet_gdf_subset


In [51]:
t0 = time.time()

six = 2000
eix = 3000

fx_df = fishnet_gdf.iloc[six:eix].copy()

fx_df['sqm_target'] = fx_df.apply(lambda x: get_data_apply(x, target_gdf), axis=1)

time.time() - t0

45.09138298034668

In [16]:
n_grids = 1000
chunks = [[i, i+n_grids] for i in range(0, fishnet_gdf.shape[0], n_grids)]

t0 = time.time()

l = []
for i in chunks[:1]:
    
#     print(i)
    
    fx = fishnet_gdf.iloc[i[0]:i[1]].copy()
    
    df = getit(fx, target_gdf)
    
    l.append(df)
    
print(time.time() - t0)
    

73.43789410591125


In [10]:
l

[         id                                           geometry  sqm_target
 0   9307735  POLYGON ((697447.164 -227065.201, 698447.164 -...         0.0
 1   9307736  POLYGON ((697447.164 -228065.201, 698447.164 -...         0.0
 2   9307737  POLYGON ((697447.164 -229065.201, 698447.164 -...         0.0
 3   9307738  POLYGON ((697447.164 -230065.201, 698447.164 -...         0.0
 4   9307739  POLYGON ((697447.164 -231065.201, 698447.164 -...         0.0
 ..      ...                                                ...         ...
 95  9307830  POLYGON ((697447.164 -322065.201, 698447.164 -...         0.0
 96  9307831  POLYGON ((697447.164 -323065.201, 698447.164 -...         0.0
 97  9307832  POLYGON ((697447.164 -324065.201, 698447.164 -...         0.0
 98  9307833  POLYGON ((697447.164 -325065.201, 698447.164 -...         0.0
 99  9307834  POLYGON ((697447.164 -326065.201, 698447.164 -...         0.0
 
 [100 rows x 3 columns],
           id                                           geome

In [61]:
# count of CPU cores in our system
n_cpus = mp.cpu_count()

# create slice intervals for the number of fishnet polygons to process at one time
n_grids = 100
chunks = [[i, i+n_grids] for i in range(0, fishnet_gdf.shape[0], n_grids)]

# construct a pool of processes where n processes == n_cores
pool = mp.Pool(processes=n_cpus)

chunk_processes = [pool.apply_async(getit, args=(fishnet_gdf.iloc[i[0]:i[1]].copy(), target_gdf)) for i in chunks[:3]]



Process ForkPoolWorker-37:
Process ForkPoolWorker-36:
Process ForkPoolWorker-35:
Process ForkPoolWorker-38:
Process ForkPoolWorker-34:
Process ForkPoolWorker-39:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python

In [62]:
[i.get() for i in chunk_processes]

KeyboardInterrupt: 